## Read and Clean Dataset


Import the libraries.


In [1]:
# Install tensorflow and keras in Anaconda
# !pip install tensorflow
# !pip install keras

In [2]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn import metrics

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Read the data into a <em>pandas</em> dataframe.


In [3]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa.


#### Let's check how many data points we have.


In [4]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors (X) and target (y)


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [7]:
concrete_data_columns = concrete_data.columns

X = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
y = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [8]:
X.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
y.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [10]:
n_cols = X.shape[1] # number of predictors
print("Number of features:", n_cols)

Number of features: 8


<a id="item1"></a>


<a id='item32'></a>


## Build a Neural Network


Let's define a function that defines our neural network model for us so that we can conveniently call it to create our model.


In [11]:
# define regression model
def create_neural_network():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has one hidden layers, each of 10 hidden units.


<a id="item4"></a>


<a id='item34'></a>


## Train and Test the Network


Let's call the function now to create our model.


In [12]:
# build the model
model = create_neural_network()

Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 100 epochs.


In [13]:
# fit the model
mean_sq_error = [] # Empty list of mean squared error

def split_and_fit_model(myModel):
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=4)
    
    myModel.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=2)
    
    # evaluate the model
    scores = myModel.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: {} \n Error: {}".format(scores, 100-scores*100))
    
    # Get the predicted values
    yhat = myModel.predict(X_test)
    mse_sk = metrics.mean_squared_error(y_test, yhat) # Calculate RMSE
    mean_sq_error.append(mse_sk)

# Step 4
for i in range(50):
    print("Iteration %s:\n" %str(i))
    split_and_fit_model(model)
    
print(len(mean_sq_error))
print("Mean:", np.mean(np.array(mean_sq_error)))
print("Standard Deviation :", np.std(np.array(mean_sq_error)))

Iteration 0:

Epoch 1/50
16/16 - 1s - loss: 3337.1255 - val_loss: 3719.6260 - 535ms/epoch - 33ms/step
Epoch 2/50
16/16 - 0s - loss: 2821.7224 - val_loss: 3208.6472 - 38ms/epoch - 2ms/step
Epoch 3/50
16/16 - 0s - loss: 2450.3745 - val_loss: 2770.4177 - 36ms/epoch - 2ms/step
Epoch 4/50
16/16 - 0s - loss: 2180.5139 - val_loss: 2424.2852 - 37ms/epoch - 2ms/step
Epoch 5/50
16/16 - 0s - loss: 1913.3873 - val_loss: 2166.1641 - 35ms/epoch - 2ms/step
Epoch 6/50
16/16 - 0s - loss: 1691.3905 - val_loss: 1894.3970 - 36ms/epoch - 2ms/step
Epoch 7/50
16/16 - 0s - loss: 1533.2555 - val_loss: 1712.7974 - 38ms/epoch - 2ms/step
Epoch 8/50
16/16 - 0s - loss: 1363.9476 - val_loss: 1516.8876 - 38ms/epoch - 2ms/step
Epoch 9/50
16/16 - 0s - loss: 1248.8595 - val_loss: 1359.4110 - 34ms/epoch - 2ms/step
Epoch 10/50
16/16 - 0s - loss: 1132.9487 - val_loss: 1220.0590 - 38ms/epoch - 2ms/step
Epoch 11/50
16/16 - 0s - loss: 1046.6863 - val_loss: 1138.6962 - 34ms/epoch - 2ms/step
Epoch 12/50
16/16 - 0s - loss: 944.5

Epoch 46/50
16/16 - 0s - loss: 110.7398 - val_loss: 103.3473 - 36ms/epoch - 2ms/step
Epoch 47/50
16/16 - 0s - loss: 113.5203 - val_loss: 140.4639 - 36ms/epoch - 2ms/step
Epoch 48/50
16/16 - 0s - loss: 114.6563 - val_loss: 123.1609 - 61ms/epoch - 4ms/step
Epoch 49/50
16/16 - 0s - loss: 100.6535 - val_loss: 104.2542 - 37ms/epoch - 2ms/step
Epoch 50/50
16/16 - 0s - loss: 100.9079 - val_loss: 102.0563 - 38ms/epoch - 2ms/step
Accuracy: 118.41254425048828 
 Error: -11741.254425048828
10/10 [==============================] - 0s 889us/step
Iteration 2:

Epoch 1/50
16/16 - 0s - loss: 101.0273 - val_loss: 102.5650 - 72ms/epoch - 4ms/step
Epoch 2/50
16/16 - 0s - loss: 100.8407 - val_loss: 100.4947 - 37ms/epoch - 2ms/step
Epoch 3/50
16/16 - 0s - loss: 101.2690 - val_loss: 101.3590 - 37ms/epoch - 2ms/step
Epoch 4/50
16/16 - 0s - loss: 100.4489 - val_loss: 101.2364 - 35ms/epoch - 2ms/step
Epoch 5/50
16/16 - 0s - loss: 105.9180 - val_loss: 108.6815 - 35ms/epoch - 2ms/step
Epoch 6/50
16/16 - 0s - loss

Epoch 40/50
16/16 - 0s - loss: 98.7783 - val_loss: 101.5569 - 37ms/epoch - 2ms/step
Epoch 41/50
16/16 - 0s - loss: 99.7888 - val_loss: 102.3857 - 34ms/epoch - 2ms/step
Epoch 42/50
16/16 - 0s - loss: 99.9691 - val_loss: 101.9500 - 35ms/epoch - 2ms/step
Epoch 43/50
16/16 - 0s - loss: 102.2114 - val_loss: 114.2591 - 35ms/epoch - 2ms/step
Epoch 44/50
16/16 - 0s - loss: 101.2271 - val_loss: 108.7986 - 34ms/epoch - 2ms/step
Epoch 45/50
16/16 - 0s - loss: 103.9770 - val_loss: 100.1469 - 34ms/epoch - 2ms/step
Epoch 46/50
16/16 - 0s - loss: 101.3279 - val_loss: 99.6523 - 35ms/epoch - 2ms/step
Epoch 47/50
16/16 - 0s - loss: 101.6854 - val_loss: 110.5789 - 36ms/epoch - 2ms/step
Epoch 48/50
16/16 - 0s - loss: 100.4910 - val_loss: 102.0385 - 34ms/epoch - 2ms/step
Epoch 49/50
16/16 - 0s - loss: 99.7535 - val_loss: 100.0281 - 36ms/epoch - 2ms/step
Epoch 50/50
16/16 - 0s - loss: 103.3018 - val_loss: 103.7282 - 36ms/epoch - 2ms/step
Accuracy: 121.3896484375 
 Error: -12038.96484375
10/10 [=============

16/16 - 0s - loss: 103.7665 - val_loss: 114.6777 - 36ms/epoch - 2ms/step
Epoch 35/50
16/16 - 0s - loss: 105.6483 - val_loss: 103.4290 - 33ms/epoch - 2ms/step
Epoch 36/50
16/16 - 0s - loss: 99.3624 - val_loss: 100.4781 - 33ms/epoch - 2ms/step
Epoch 37/50
16/16 - 0s - loss: 100.7102 - val_loss: 100.3582 - 36ms/epoch - 2ms/step
Epoch 38/50
16/16 - 0s - loss: 99.6850 - val_loss: 104.3624 - 36ms/epoch - 2ms/step
Epoch 39/50
16/16 - 0s - loss: 99.2002 - val_loss: 104.0911 - 36ms/epoch - 2ms/step
Epoch 40/50
16/16 - 0s - loss: 100.3334 - val_loss: 101.1373 - 37ms/epoch - 2ms/step
Epoch 41/50
16/16 - 0s - loss: 100.0462 - val_loss: 100.9153 - 34ms/epoch - 2ms/step
Epoch 42/50
16/16 - 0s - loss: 100.1133 - val_loss: 101.6190 - 32ms/epoch - 2ms/step
Epoch 43/50
16/16 - 0s - loss: 99.9122 - val_loss: 106.4131 - 36ms/epoch - 2ms/step
Epoch 44/50
16/16 - 0s - loss: 107.4814 - val_loss: 110.2523 - 37ms/epoch - 2ms/step
Epoch 45/50
16/16 - 0s - loss: 114.0254 - val_loss: 110.1262 - 37ms/epoch - 2ms/s

16/16 - 0s - loss: 101.1833 - val_loss: 109.3366 - 34ms/epoch - 2ms/step
Epoch 29/50
16/16 - 0s - loss: 104.0698 - val_loss: 105.0005 - 34ms/epoch - 2ms/step
Epoch 30/50
16/16 - 0s - loss: 106.7549 - val_loss: 100.5259 - 33ms/epoch - 2ms/step
Epoch 31/50
16/16 - 0s - loss: 102.4576 - val_loss: 108.1667 - 32ms/epoch - 2ms/step
Epoch 32/50
16/16 - 0s - loss: 102.8076 - val_loss: 104.6401 - 35ms/epoch - 2ms/step
Epoch 33/50
16/16 - 0s - loss: 103.2277 - val_loss: 118.7365 - 38ms/epoch - 2ms/step
Epoch 34/50
16/16 - 0s - loss: 101.3454 - val_loss: 101.7321 - 37ms/epoch - 2ms/step
Epoch 35/50
16/16 - 0s - loss: 98.1914 - val_loss: 103.0245 - 37ms/epoch - 2ms/step
Epoch 36/50
16/16 - 0s - loss: 101.9641 - val_loss: 110.6148 - 38ms/epoch - 2ms/step
Epoch 37/50
16/16 - 0s - loss: 105.2893 - val_loss: 100.3820 - 36ms/epoch - 2ms/step
Epoch 38/50
16/16 - 0s - loss: 103.8106 - val_loss: 102.2393 - 36ms/epoch - 2ms/step
Epoch 39/50
16/16 - 0s - loss: 101.1528 - val_loss: 99.9952 - 35ms/epoch - 2ms

...

16/16 - 0s - loss: 51.5771 - val_loss: 63.3485 - 35ms/epoch - 2ms/step
Epoch 43/50
16/16 - 0s - loss: 51.6141 - val_loss: 50.3992 - 39ms/epoch - 2ms/step
Epoch 44/50
16/16 - 0s - loss: 50.7473 - val_loss: 51.5711 - 38ms/epoch - 2ms/step
Epoch 45/50
16/16 - 0s - loss: 53.8721 - val_loss: 46.8466 - 33ms/epoch - 2ms/step
Epoch 46/50
16/16 - 0s - loss: 49.8886 - val_loss: 47.0244 - 34ms/epoch - 2ms/step
Epoch 47/50
16/16 - 0s - loss: 51.2033 - val_loss: 46.5647 - 32ms/epoch - 2ms/step
Epoch 48/50
16/16 - 0s - loss: 51.0342 - val_loss: 46.2068 - 36ms/epoch - 2ms/step
Epoch 49/50
16/16 - 0s - loss: 50.2528 - val_loss: 46.4787 - 33ms/epoch - 2ms/step
Epoch 50/50
16/16 - 0s - loss: 49.9964 - val_loss: 47.3850 - 34ms/epoch - 2ms/step
Accuracy: 51.01714324951172 
 Error: -5001.714324951172
10/10 [==============================] - 0s 1ms/step
Iteration 42:

Epoch 1/50
16/16 - 0s - loss: 50.6982 - val_loss: 47.8512 - 81ms/epoch - 5ms/step
Epoch 2/50
16/16 - 0s - loss: 50.4678 - val_loss: 46.7319 -

16/16 - 0s - loss: 50.5423 - val_loss: 45.8188 - 34ms/epoch - 2ms/step
Epoch 39/50
16/16 - 0s - loss: 50.6787 - val_loss: 57.2737 - 34ms/epoch - 2ms/step
Epoch 40/50
16/16 - 0s - loss: 50.1843 - val_loss: 48.6147 - 37ms/epoch - 2ms/step
Epoch 41/50
16/16 - 0s - loss: 51.9683 - val_loss: 51.4662 - 35ms/epoch - 2ms/step
Epoch 42/50
16/16 - 0s - loss: 53.7540 - val_loss: 61.3532 - 37ms/epoch - 2ms/step
Epoch 43/50
16/16 - 0s - loss: 52.4411 - val_loss: 45.2720 - 34ms/epoch - 2ms/step
Epoch 44/50
16/16 - 0s - loss: 48.9367 - val_loss: 49.4558 - 33ms/epoch - 2ms/step
Epoch 45/50
16/16 - 0s - loss: 48.7964 - val_loss: 45.6177 - 32ms/epoch - 2ms/step
Epoch 46/50
16/16 - 0s - loss: 49.1991 - val_loss: 45.8925 - 33ms/epoch - 2ms/step
Epoch 47/50
16/16 - 0s - loss: 49.5793 - val_loss: 45.5590 - 36ms/epoch - 2ms/step
Epoch 48/50
16/16 - 0s - loss: 50.1795 - val_loss: 45.5683 - 33ms/epoch - 2ms/step
Epoch 49/50
16/16 - 0s - loss: 49.9836 - val_loss: 45.4102 - 34ms/epoch - 2ms/step
Epoch 50/50
16/1

16/16 - 0s - loss: 49.2925 - val_loss: 49.0840 - 34ms/epoch - 2ms/step
Epoch 35/50
16/16 - 0s - loss: 50.1448 - val_loss: 47.2112 - 34ms/epoch - 2ms/step
Epoch 36/50
16/16 - 0s - loss: 50.5626 - val_loss: 53.2323 - 32ms/epoch - 2ms/step
Epoch 37/50
16/16 - 0s - loss: 50.8827 - val_loss: 53.7127 - 33ms/epoch - 2ms/step
Epoch 38/50
16/16 - 0s - loss: 49.6768 - val_loss: 45.7621 - 35ms/epoch - 2ms/step
Epoch 39/50
16/16 - 0s - loss: 51.9572 - val_loss: 48.1108 - 35ms/epoch - 2ms/step
Epoch 40/50
16/16 - 0s - loss: 55.0750 - val_loss: 45.4277 - 38ms/epoch - 2ms/step
Epoch 41/50
16/16 - 0s - loss: 51.7264 - val_loss: 46.2156 - 34ms/epoch - 2ms/step
Epoch 42/50
16/16 - 0s - loss: 50.2124 - val_loss: 63.0329 - 37ms/epoch - 2ms/step
Epoch 43/50
16/16 - 0s - loss: 54.2169 - val_loss: 59.4012 - 38ms/epoch - 2ms/step
Epoch 44/50
16/16 - 0s - loss: 59.6144 - val_loss: 54.3490 - 33ms/epoch - 2ms/step
Epoch 45/50
16/16 - 0s - loss: 51.7525 - val_loss: 47.3551 - 32ms/epoch - 2ms/step
Epoch 46/50
16/1

16/16 - 0s - loss: 48.8783 - val_loss: 45.3566 - 33ms/epoch - 2ms/step
Epoch 31/50
16/16 - 0s - loss: 48.5363 - val_loss: 51.1563 - 33ms/epoch - 2ms/step
Epoch 32/50
16/16 - 0s - loss: 48.2529 - val_loss: 57.1436 - 34ms/epoch - 2ms/step
Epoch 33/50
16/16 - 0s - loss: 53.3461 - val_loss: 45.7494 - 36ms/epoch - 2ms/step
Epoch 34/50
16/16 - 0s - loss: 49.8354 - val_loss: 53.4726 - 36ms/epoch - 2ms/step
Epoch 35/50
16/16 - 0s - loss: 49.3882 - val_loss: 45.2886 - 35ms/epoch - 2ms/step
Epoch 36/50
16/16 - 0s - loss: 48.5990 - val_loss: 46.7475 - 35ms/epoch - 2ms/step
Epoch 37/50
16/16 - 0s - loss: 48.2803 - val_loss: 45.0662 - 35ms/epoch - 2ms/step
Epoch 38/50
16/16 - 0s - loss: 49.5348 - val_loss: 48.2023 - 36ms/epoch - 2ms/step
Epoch 39/50
16/16 - 0s - loss: 49.0529 - val_loss: 47.9347 - 37ms/epoch - 2ms/step
Epoch 40/50
16/16 - 0s - loss: 49.4225 - val_loss: 48.5967 - 33ms/epoch - 2ms/step
Epoch 41/50
16/16 - 0s - loss: 48.4338 - val_loss: 45.1448 - 32ms/epoch - 2ms/step
Epoch 42/50
16/1

16/16 - 0s - loss: 49.4967 - val_loss: 45.8553 - 34ms/epoch - 2ms/step
Epoch 27/50
16/16 - 0s - loss: 50.2205 - val_loss: 44.8242 - 32ms/epoch - 2ms/step
Epoch 28/50
16/16 - 0s - loss: 49.9268 - val_loss: 57.0876 - 36ms/epoch - 2ms/step
Epoch 29/50
16/16 - 0s - loss: 49.3968 - val_loss: 49.9679 - 34ms/epoch - 2ms/step
Epoch 30/50
16/16 - 0s - loss: 48.3475 - val_loss: 46.2993 - 34ms/epoch - 2ms/step
Epoch 31/50
16/16 - 0s - loss: 49.2514 - val_loss: 45.5196 - 35ms/epoch - 2ms/step
Epoch 32/50
16/16 - 0s - loss: 48.2616 - val_loss: 45.1926 - 34ms/epoch - 2ms/step
Epoch 33/50
16/16 - 0s - loss: 49.2572 - val_loss: 48.2619 - 33ms/epoch - 2ms/step
Epoch 34/50
16/16 - 0s - loss: 48.8846 - val_loss: 58.1362 - 33ms/epoch - 2ms/step
Epoch 35/50
16/16 - 0s - loss: 49.5268 - val_loss: 45.9316 - 35ms/epoch - 2ms/step
Epoch 36/50
16/16 - 0s - loss: 48.1357 - val_loss: 46.9609 - 34ms/epoch - 2ms/step
Epoch 37/50
16/16 - 0s - loss: 49.6966 - val_loss: 45.1720 - 35ms/epoch - 2ms/step
Epoch 38/50
16/1